In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [24]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor

spark = SparkSession.builder.appName("Spark ML").getOrCreate()

evDF = spark.read.option("inferSchema","true").option("header","true").csv("/content/drive/MyDrive/EuroCC/WinterSchool/ev-fiyatlari.txt")
evDF.show()
evDF = evDF.withColumnRenamed("Fiyat","label")
vec  = VectorAssembler(inputCols=["OdaSayisi","SalonSayisi",
                                  "KacMetreKare","BalkonVar",
                                  "BanyoSayisi","KacinciKat"],
                       outputCol="features")

evDF = vec.transform(evDF)
evDF = evDF.select("features","label")
evDF.show()
regressor = DecisionTreeRegressor()
model = regressor.fit(evDF) #Model Training
model.save("abc")
print("Model Hesaplandı")


+---------+-----------+------------+---------+-----------+----------+------+
|OdaSayisi|SalonSayisi|KacMetreKare|BalkonVar|BanyoSayisi|KacinciKat| Fiyat|
+---------+-----------+------------+---------+-----------+----------+------+
|        3|          1|         130|        1|          1|         4|250000|
|        2|          1|          80|        1|          1|         1|100000|
|        4|          2|         200|        1|          2|         2|500000|
|        4|          1|         120|        1|          1|         8|300000|
|        1|          1|          70|        1|          1|         9| 90000|
+---------+-----------+------------+---------+-----------+----------+------+

+--------------------+------+
|            features| label|
+--------------------+------+
|[3.0,1.0,130.0,1....|250000|
|[2.0,1.0,80.0,1.0...|100000|
|[4.0,2.0,200.0,1....|500000|
|[4.0,1.0,120.0,1....|300000|
|[1.0,1.0,70.0,1.0...| 90000|
+--------------------+------+

Model Hesaplandı


In [23]:
yeniEvlerDF = spark.read.option("inferSchema","true").option("header","true").csv("/content/drive/MyDrive/EuroCC/WinterSchool/yeni-evler.txt")

yeniEvlerDF = vec.transform(yeniEvlerDF)
yeniEvlerDF.show()
sonucDF = model.transform(yeniEvlerDF)
sonucDF.show()
#Data Visualization matplotlib, seaborn kütüphanelerini kullamamız gerekiyor

+---------+-----------+------------+---------+-----------+----------+--------------------+
|OdaSayisi|SalonSayisi|KacMetreKare|BalkonVar|BanyoSayisi|KacinciKat|            features|
+---------+-----------+------------+---------+-----------+----------+--------------------+
|        3|          1|         128|        1|          1|         1|[3.0,1.0,128.0,1....|
|        2|          1|          88|        0|          1|         5|[2.0,1.0,88.0,0.0...|
+---------+-----------+------------+---------+-----------+----------+--------------------+

+---------+-----------+------------+---------+-----------+----------+--------------------+----------+
|OdaSayisi|SalonSayisi|KacMetreKare|BalkonVar|BanyoSayisi|KacinciKat|            features|prediction|
+---------+-----------+------------+---------+-----------+----------+--------------------+----------+
|        3|          1|         128|        1|          1|         1|[3.0,1.0,128.0,1....|  250000.0|
|        2|          1|          88|        0

In [17]:
print(model.coefficients)
print(model.intercept)

[42571.42866130554,48000.00027696489,1771.4285662124746,0.0,48000.00027696489,6285.714320407779]
-229142.8575998255


In [18]:
def evFiyatTahmin(OdaSayisi,SalonSayisi,KacMetreKare,BalkonVar,BanyoSayisi,KacinciKat):
  return 42571.42866130554 * OdaSayisi + 48000.00027696489*SalonSayisi+1771.4285662124746*KacMetreKare+ 0.0*BalkonVar+48000.00027696489*BanyoSayisi + 6285.714320407779*KacinciKat + -229142.8575998255

In [19]:
evFiyatTahmin(3,1,128,1,1,1)

227599.99973362542

In [ ]:
from pyspark.ml.regression import   DecisionTreeRegressionModel
from pyspark.ml.classification import MultilayerPerceptronClassifier
mymodel = DecisionTreeRegressionModel.load("abc")
mymodel.transform(yeniEvlerDF)